In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure

In [ ]:
#removing space after authors 
def fix_author(row):
    if not isinstance(row, str) or row[-1] != ' ':
        return row
    return row[:-1]

In [ ]:
#This is path to output csv files of the index of each book with they italicized names filtered out
vol1_path = '../input/index_without_itallics_csvs/vol1/vol1_index_pruned.csv'
vol2_path = '../input/index_without_itallics_csvs/vol2/vol2_index_pruned.csv'
vol3_path = '../input/index_without_itallics_csvs/vol3/vol3_index_pruned.csv'

vol1_df = pd.read_csv(vol1_path)
vol2_df = pd.read_csv(vol2_path)
vol3_df = pd.read_csv(vol3_path)

#fix spacing issue
vol1_df['author'] = vol1_df['author'].apply(fix_author)
vol2_df['author'] = vol2_df['author'].apply(fix_author)
vol3_df['author'] = vol3_df['author'].apply(fix_author)

#changing name of columns 
vol1_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol2_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol3_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)

#wcvp 
text_wcvp_path = '../input/wcvp/wcvp_v9_jun_2022.txt'
wcvp_df = pd.read_csv(text_wcvp_path, sep = '|')

# PlayGround_1007

In [ ]:
vol1_df

In [ ]:
#concat all index volumes into one #not sure if this is a good idea or bad one should check runtime ... will deal with it later :D

In [ ]:
vol1_df

In [ ]:
new_df = pd.merge(vol1_df, wcvp_df,  how='left', left_on=['mouterde_genus','mouterde_epithet'], right_on = ['genus','species'])

In [ ]:
new_df.shape

In [ ]:
new_df.iloc[:,:17]

In [ ]:
# excluding infra as vol1 doesn't have infra species (need to figure out how to deal with null values here) 
# Need to drop genus only entries as we don't have genus author for volumes 1 and 2 and can't match them with correct author 

no_genus_only_no_infra_vol1_df = vol1_df[~((vol1_df['mouterde_epithet']).isnull()) & ((vol1_df['mouterde_infra']).isnull())] 
no_infra_wcvp_df = wcvp_df[(wcvp_df['infraspecies']).isnull()]
species_only_df = pd.merge(no_genus_only_no_infra_vol1_df, no_infra_wcvp_df,  how='inner', 
                            left_on=['mouterde_genus','mouterde_epithet', 'mouterde_author'], 
                            right_on = ['genus','species', 'authors'])

In [ ]:
species_only_df['authors'][0]

In [ ]:
species_only_df['mouterde_author'][0]

In [ ]:
species_only_df.iloc[:,5:20]

In [ ]:
#expect to get 0 if everything is matched correctly --- nvm doing something different now 
vol1_df.shape[0] - vol1_df['mouterde_genus'].unique().shape[0] - species_only_df.shape[0]

In [ ]:
species_only_df[['mouterde_genus', 'mouterde_epithet', 'mouterde_author', 'taxonomic_status', 'genus', 'species', 'authors']]

In [ ]:
with_accepted_name = species_only_df[['mouterde_genus', 'mouterde_epithet', 'mouterde_author', 'taxonomic_status', 'accepted_name', 'accepted_authors']]

In [ ]:
with_accepted_name[with_accepted_name['taxonomic_status'] == 'Synonym']

In [ ]:
species_only_df

# Exact Matching

In [ ]:
wcvp_df['infraspecies'].replace(np.nan, 'np.NaN', inplace=True)
wcvp_df['species'].replace(np.nan, 'np.NaN', inplace=True)

## Vol1 

In [ ]:
vol1_df['mouterde_infra'].replace(np.nan, 'np.NaN', inplace=True)
vol1_df['mouterde_epithet'].replace(np.nan, 'np.NaN', inplace=True)

vol1_wcvp_df = pd.merge(vol1_df, 
                        wcvp_df, 
                        how='inner', 
                        left_on=['mouterde_genus','mouterde_epithet', 'mouterde_author', 'mouterde_infra'], 
                        right_on = ['genus','species', 'authors', 'infraspecies'])
vol1_wcvp_df.replace('np.NaN', np.nan, inplace=True)

In [ ]:
vol1_wcvp_df_simplified = vol1_wcvp_df[['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author', 'taxonomic_status', 'accepted_name', 'accepted_authors']]
# output1 = '../output/fuzzy_matching/index/exact_matching/vol1_index_noItalics_wcvp.csv'
# vol1_wcvp_df_simplified.to_csv(output1)

In [ ]:
#percentage picked up with exact matching
vol1_wcvp_df_simplified.shape[0]/vol1_df.shape[0]

In [ ]:
#percentage of picked up species with exact matching that are accepted 
#.sum == .value_counts()[True] here
(vol1_wcvp_df_simplified['taxonomic_status'] == 'Accepted').sum()/vol1_wcvp_df_simplified.shape[0]

## Vol2

In [ ]:
vol2_df['mouterde_infra'].replace(np.nan, 'np.NaN', inplace=True)
vol2_df['mouterde_epithet'].replace(np.nan, 'np.NaN', inplace=True)

vol2_wcvp_df = pd.merge(vol2_df, 
                        wcvp_df, 
                        how='inner', 
                        left_on=['mouterde_genus','mouterde_epithet', 'mouterde_author', 'mouterde_infra'], 
                        right_on = ['genus','species', 'authors', 'infraspecies'])
vol2_wcvp_df.replace('np.NaN', np.nan, inplace=True)

In [ ]:
vol2_wcvp_df_simplified = vol2_wcvp_df[['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author', 'taxonomic_status', 'accepted_name', 'accepted_authors']]
output2 = '../output/fuzzy_matching/index/exact_matching/vol2_index_noItalics_wcvp.csv'
vol2_wcvp_df_simplified.to_csv(output2)

In [ ]:
#percentage picked up with exact matching
vol2_wcvp_df_simplified.shape[0]/vol2_df.shape[0]

In [ ]:
#percentage of picked up species with exact matching that are accepted 
#.sum == .value_counts()[True] here
(vol2_wcvp_df_simplified['taxonomic_status'] == 'Accepted').sum()/vol2_wcvp_df_simplified.shape[0]

## Vol3

In [ ]:
vol3_df['mouterde_infra'].replace(np.nan, 'np.NaN', inplace=True)
vol3_df['mouterde_epithet'].replace(np.nan, 'np.NaN', inplace=True)

vol3_wcvp_df = pd.merge(vol3_df, 
                        wcvp_df, 
                        how='inner', 
                        left_on=['mouterde_genus','mouterde_epithet', 'mouterde_author', 'mouterde_infra'], 
                        right_on = ['genus','species', 'authors', 'infraspecies'])

vol3_wcvp_df.replace('np.NaN', np.nan, inplace=True)

In [ ]:
vol3_wcvp_df_simplified = vol3_wcvp_df[['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author', 'taxonomic_status', 'accepted_name', 'accepted_authors']]
output3 = '../output/fuzzy_matching/index/exact_matching/vol3_index_noItalics_wcvp.csv'
vol3_wcvp_df_simplified.to_csv(output3)

In [ ]:
#percentage picked up with exact matching
vol3_wcvp_df_simplified.shape[0]/vol3_df.shape[0]

In [ ]:
#percentage of picked up species with exact matching that are accepted 
#.sum == .value_counts()[True] here
(vol3_wcvp_df_simplified['taxonomic_status'] == 'Accepted').sum()/vol3_wcvp_df_simplified.shape[0]

## Exact matiching Graph

In [ ]:
pick_up_rates = [vol1_wcvp_df_simplified.shape[0]/vol1_df.shape[0],
                 vol2_wcvp_df_simplified.shape[0]/vol2_df.shape[0],
                 vol3_wcvp_df_simplified.shape[0]/vol3_df.shape[0]]
accepted_rate = [(vol1_wcvp_df_simplified['taxonomic_status'] == 'Accepted').sum()/vol1_df.shape[0],
                 (vol2_wcvp_df_simplified['taxonomic_status'] == 'Accepted').sum()/vol2_df.shape[0],
                 (vol3_wcvp_df_simplified['taxonomic_status'] == 'Accepted').sum()/vol3_df.shape[0]]

ax = plt.subplots()

pastel_blue = sns.color_palette("pastel")[0]
pastel_red = sns.color_palette("pastel")[3]

ax = sns.barplot(x=['vol1', 'vol2', 'vol3'], y=pick_up_rates, color=pastel_blue, label = 'percentage picked up')
ax = sns.barplot(x=['vol1', 'vol2', 'vol3'], y=accepted_rate, color=pastel_red, label = 'percentage accepted')

ax.set(xlabel="volume", ylabel="percentage", title = "performance using exact matching")

plt.legend()
plt.show()

# Playground 1013

In [ ]:
vol1_df['mouterde_infra'].replace(np.nan, 'np.NaN', inplace=True)
vol1_df['mouterde_epithet'].replace(np.nan, 'np.NaN', inplace=True)

vol1_wcvp_df = pd.merge(vol1_df, 
                        wcvp_df, 
                        how='left', 
                        left_on=['mouterde_genus','mouterde_epithet', 'mouterde_author', 'mouterde_infra'], 
                        right_on = ['genus','species', 'authors', 'infraspecies'])
vol1_wcvp_df.replace('np.NaN', np.nan, inplace=True)

In [ ]:
vol1_wcvp_df_mismatch = vol1_wcvp_df[(vol1_wcvp_df['kew_id'].isnull()) & ~(vol1_wcvp_df['mouterde_author'].isnull()) ]
vol1_wcvp_df_mismatch[['page_num', 'mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author']]

In [ ]:
wcvp_df['authors'].unique()

In [ ]:
unique_authors = vol1_df[vol1_df['mouterde_author'].notnull()]['mouterde_author'].unique()

In [ ]:
unique_authors.sort()

In [ ]:
unique_authors

In [ ]:
Vulpia	inops	(Del.) Hack.
Vulpia	inops 	Hack.


Vulpia	membranacea	(L.) Link
Vulpia	membranacea	(L.) Dumort.

In [ ]:
species_only_df_vol1.columns

In [ ]:
vol1_wcvp_df_simplified = vol1_wcvp_df[['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author', 'taxonomic_status', 'accepted_name', 'accepted_authors']]
output1 = '../output/fuzzy_matching/index/exact_matching/vol1_index_noItalics_wcvp.csv'
vol1_wcvp_df_simplified.to_csv(output1)